<a href="https://colab.research.google.com/github/udoyroy98/Motor_Fault_Detection_ML/blob/main/SVM_Motor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import matplotlib.pyplot as plt
from google.colab import drive
import seaborn as snNew
import pandas as pdNew
import matplotlib.pyplot as pltNew


In [ ]:
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/MyDrive/Dataset/0-5train.csv')


In [ ]:
x = data.iloc[:,:-1]
y = data.iloc[:,9]


In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.25 , random_state=42)
#normalize
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
xtrain = sc.fit_transform(xtrain)
xtest = sc.transform(xtest)


In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from sklearn import svm
krn = ['linear','poly','rbf','sigmoid']
rng_C = np.arange(1,5,7)
rng_deg = np.arange(3,5,7)
rng_co = np.arange(0.001,10,0.5)
rng_gam = ['auto','scale']
from sklearn.metrics import accuracy_score
best_score=0
for i in krn:
  for j in rng_C:
    for k in rng_deg:
      for z in rng_co:
        for x in rng_gam:
          SVModel=svm.SVC(kernel=i,C=j,degree=k,coef0=z,gamma=x)
          SVModel.fit(xtrain, ytrain)
          acc_score=accuracy_score(ytest,SVModel.predict(xtest))
          if best_score<acc_score:
            best_score=acc_score
            bi=i
            bj=j
            bk=k
            bz=z
            bx=x
print(best_score,bi,bj,bk,bz,bx)


In [ ]:
model4 = OneVsRestClassifier(BaggingClassifier(svm.SVC(kernel = 'poly',degree=3,coef0=5.001,probability=True,gamma='auto',C=1)))
model4.fit(xtrain, ytrain) #train model
y_pred4 = model4.predict(xtest)
print(classification_report(ytest, y_pred4)) #accuracy


In [ ]:
from sklearn.preprocessing import label_binarize
classes = np.unique(ytest)
ytest_binarized = label_binarize(ytest,classes = np.unique(ytest))
fpr = {}
tpr = {}
thresh = {}
roc_auc = dict()
n_class = classes.shape[0]
for i in range(n_class):
  fpr[i],tpr[i],thresh[i] = roc_curve(ytest_binarized[:,i],pred_prob4[:,i])
  roc_auc[i] = auc(fpr[i],tpr[i])


In [ ]:
#plotting
plt.plot(fpr[0], tpr[0], linestyle='--',color='orange', label='Class 0 vs Rest(AUC=%0.2f)'%(roc_auc[0]))
plt.plot(fpr[1], tpr[1], linestyle='--',color='green', label='Class 1 vs Rest(AUC=%0.2f)'%(roc_auc[1]))
plt.plot(fpr[2], tpr[2], linestyle='--',color='blue', label='Class 2 vs Rest(AUC=%0.2f)'%(roc_auc[2]))
plt.plot([0,1],[0,1],'r--')
plt.xlim([0,1])
plt.ylim([0,1.05])
plt.title('Multiclass ROC curve')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.legend(loc = 'Lower right')
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix
c_mat4 = confusion_matrix(ytest, y_pred4)
c_mat4
